In [8]:
!ls

customized_linear.py		train_vega_kang_control.py
learning_utils.py		train_vega_kang_pbmc.py
pathway_deep_decoder_vae.py	train_vega_mixseq.py
pathway_sparse_vae.py		train_vega_organoid.py
process_mixseq.py		train_vega_suppFig1.py
__pycache__			Untitled0.ipynb
train_deep_decoder_suppFig1.py	utils.py
train_sparse_vae_suppFig1.py	vanilla_vae.py
train_vanilla_vae_suppFig1.py	vega_model.py
train_vega_ct_out_pbmc.py	world_model.ipynb
train_vega_gbm.py


In [7]:
# Google Colaboratoryで作業する場合はこちらも実行してください。
from google.colab import drive
drive.mount('/content/drive')
# %cd 以降にこのnotebookを置いているディレクトリを指定してください。
%cd "/content/drive/MyDrive/UT_world_model/Worldmodel/vega-reproducibility/src"

Mounted at /content/drive
/content/drive/MyDrive/UT_world_model/Worldmodel/vega-reproducibility/src


In [10]:
!pip install scanpy

     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 96 kB 4.6 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 1.1 MB 34.4 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82708 sha256=2d0fda6e91621971bab63b9c7e250f53cbde2025f165a0eb74efeef0556b3c00
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=4567f0c13cf883c020e8a274c2d7ecec3f8d837f87c576651619db2caa77bd54
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80d72741e400345b5a5b50ec3d929aca581bf45e0225d5c50
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8048 sha256=b7d343c54f96ffd656302e99950c6349d84d7a4a0143f92d9a79c981eb1e1e99
  Stored in directory: /

In [11]:
#!/usr/bin/env python3

import torch
from vega_model import VEGA
from utils import *
from learning_utils import *
import scanpy as sc
from scipy import sparse
from sklearn import preprocessing
import numpy as np
import itertools
import argparse


def train_vega():
    """ Main """
    train_path = "../data/train_pbmc.h5ad"
    pathway_file = "../data/reactomes.gmt"
    LR = 1e-4
    N_EPOCHS=500
    p_drop = 0.5

    # Set model
    random_seed = 1
    # Out dir
    local_out = './trained_models/kang_pbmc/'
    torch.backends.cudnn.enabled = True
    torch.manual_seed(random_seed)
    dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', dev, flush=True)
    dir_name = '.'+local_out
    # Read pathway file and data
    train_data = sc.read(train_path)
    labels = train_data.obs['cell_type']
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    y = torch.Tensor(le.transform(labels))
    pathway_dict = read_gmt(pathway_file, min_g=0, max_g=1000)
    print(train_data.shape, flush=True)
    pathway_mask = create_pathway_mask(train_data.var.index.tolist(), pathway_dict, add_missing=1, fully_connected=True)
    if sparse.issparse(train_data.X):
        train_ds = train_data.X.A
    else:
        train_ds = train_data.X
    train_ds = torch.Tensor(train_ds)
    train_ds = UnsupervisedDataset(train_ds, targets=y)
    # Initialize CV
    kfold = KFoldTorch(cv=10, n_epochs=N_EPOCHS, lr=LR, train_p=10, test_p=10, num_workers=0, save_all=True, save_best=False, path_dir=dir_name, model_prefix='vega_kang_pbmc_')
    dict_params = {'pathway_mask':pathway_mask, 'beta':0.00005, 'dropout':p_drop, 'path_model':None, 'device':dev, 'positive_decoder':True}
    kfold.train_kfold(VEGA, dict_params, train_ds, batch_size=64)
    np.save(dir_name+'10CV_vega_kang_pbmc.npy', kfold.cv_res_dict)
    return

if __name__=="__main__":
    train_vega()


Using device: cuda


/usr/local/lib/python3.7/dist-packages/anndata/compat/__init__.py:235: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/anndata/compat/__init__.py:235: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  FutureWarning,


(16893, 6998)
Model(s) will be saved at ../trained_models/kang_pbmc/ using vega_kang_pbmc_ as prefix
0.5
Constraining decoder to positive weights
cuda
Training fold 0
[Epoch 1] | loss: 305.535 | test_loss: 236.922 |
[Epoch 2] | loss: 230.126 | test_loss: 208.542 |
[Epoch 3] | loss: 219.857 | test_loss: 200.131 |
[Epoch 4] | loss: 215.881 | test_loss: 196.842 |
[Epoch 5] | loss: 213.386 | test_loss: 193.929 |
[Epoch 6] | loss: 211.318 | test_loss: 192.470 |
[Epoch 7] | loss: 209.602 | test_loss: 190.048 |
[Epoch 8] | loss: 208.675 | test_loss: 188.908 |
[Epoch 9] | loss: 207.474 | test_loss: 187.394 |
[Epoch 10] | loss: 205.845 | test_loss: 186.889 |
[Epoch 11] | loss: 205.813 | test_loss: 186.355 |
[Epoch 12] | loss: 204.402 | test_loss: 184.925 |
[Epoch 13] | loss: 203.642 | test_loss: 184.996 |
[Epoch 14] | loss: 202.909 | test_loss: 183.774 |
[Epoch 15] | loss: 202.092 | test_loss: 183.537 |
[Epoch 16] | loss: 201.136 | test_loss: 183.200 |
[Epoch 17] | loss: 200.598 | test_loss: 18